In [13]:
import pandas as pd
import re
from preprocessor import Preprocessor

### preprocessing_cpu 의 일부 함수들 재사용
- 주피터 노트북 파일로 작성되어, import 대신 함수 코드들을 재사용
- 재사용 함수들은 벤치마크 파일로부터 cpu 이름들을 전처리하는데 사용
- 예) Intel Core i5-10400F @ 2.90GHz -> i5-10400F

In [7]:
def get_i3_to_i9_intel_cpu(file, target_column):
    preprocessor = Preprocessor(file)
    processed = pd.DataFrame()
    for core in ['i3', 'i5', 'i7', 'i9']:
        processed = pd.concat(
            [
                processed,
                preprocessor.extract_rows_by_keyword_contains(keyword=core, target_column=target_column)
            ],
            ignore_index=True
        )
    return processed
    
def parse_intel_cpu_name(cpu_name):
    repeated = 'Intel Core'
    start_idx = cpu_name.find(repeated) + len(repeated)
    end_idx = cpu_name.find('@')
    if end_idx != -1:
        return cpu_name[start_idx:end_idx].strip()
    return cpu_name[start_idx:].strip()

def get_ryzen3_to_9_amd_cpu(file, target_column):
    preprocessor = Preprocessor(file)
    processed = pd.DataFrame()
    for core in ['3', '5', '7', '9']:
        processed = pd.concat(
            [
                processed,
                preprocessor.extract_rows_by_keyword_contains(keyword=f"Ryzen {core}", target_column=target_column)
            ],
            ignore_index=True
        )
    return processed


def parse_ryzen_cpu_name(cpu_name):
    repeated = 'AMD'
    start_idx = cpu_name.find(repeated) + len(repeated)
    return cpu_name[start_idx:].strip()

### 벤치 마크 파일 내용 중, cpu 관련 항목 전처리하기
- cpu 이름을 기존 analytics.summary 테이블에 저장되어있는 형태로 파싱하고,
- analytics.summary 테이블에 있는 제품에 해당하는 벤치마크 데이터만 추출하여 데이터 프레임으로 반환

In [8]:
def preprocess_cpu_benchmark(benchmark_file, product_names_in_db):
    """
    벤치 마크 파일 내용 중, cpu 관련 항목을 전처리한다.
    - cpu 이름을 기존 analytics.summary 테이블에 저장되어있는 형태로 파싱하고,
    - analytics.summary 테이블에 있는 제품에 해당하는 벤치마크 데이터만 추출하여 데이터 프레임으로 반환
    :param benchmark_file: passmark로부터 크롤링한 벤치마크 파일 제품이름, 벤치마크 점수, 부품 타입을 의미하는 컬럼으로 구성
    :param product_names_in_db: analytics.summary 테이블에 있는 제품이름 목록 데이터프레임
    :return:
    """
    cpu_names_in_db = product_names_in_db[product_names_in_db['components_of_computer'.upper()] == 'CPU']
    benchmarks = pd.concat([
        get_i3_to_i9_intel_cpu(benchmark_file, 'product_name'),
        get_ryzen3_to_9_amd_cpu(benchmark_file, 'product_name')
    ], ignore_index=True)
    benchmarks = benchmarks[benchmarks['components_of_computer'] == 'CPU']
    new = pd.DataFrame(columns=[
        'product_name', 'product_performance', 'components_of_computer'
    ])
    for row in benchmarks.itertuples():
        cpu_name = parse_intel_cpu_name(row.product_name) \
            if row.product_name.startswith('Intel') else parse_ryzen_cpu_name(row.product_name)
        cpu_name_matched = cpu_names_in_db[cpu_names_in_db['product_name'.upper()] == cpu_name]
        if len(cpu_name_matched) == 0:
            continue
        new.loc[len(new)] = [
            *cpu_name_matched['product_name'.upper()].values, row.product_performance, 'CPU'
        ]
    return new

### 벤치 마크 파일 내용 중, gpu 관련 항목 전처리하기
- gpu 이름을 기존 analytics.summary 테이블에 저장되어있는 형태로 파싱하고,
- analytics.summary 테이블에 있는 제품에 해당하는 벤치마크 데이터만 추출하여 데이터 프레임으로 반환

In [9]:
def preprocess_gpu_benchmark(benchmark_file, product_names_in_db):
    """
    벤치 마크 파일 내용 중, gpu 관련 항목을 전처리한다.
    - gpu 이름을 기존 analytics.summary 테이블에 저장되어있는 형태로 파싱하고,
    - analytics.summary 테이블에 있는 제품에 해당하는 벤치마크 데이터만 추출하여 데이터 프레임으로 반환
    :param benchmark_file: passmark로부터 크롤링한 벤치마크 파일 제품이름, 벤치마크 점수, 부품 타입을 의미하는 컬럼으로 구성
    :param product_names_in_db: analytics.summary 테이블에 있는 제품이름 목록 데이터프레임
    :return: 
    """
    gpu_names_in_db = product_names_in_db[product_names_in_db['components_of_computer'.upper()] == 'gpu']
    benchmarks = pd.read_csv(benchmark_file)
    benchmarks = benchmarks[benchmarks['components_of_computer'] == 'gpu']
    benchmarks['processed_name'] = benchmarks.apply(preprocess_gpu_names, axis=1)
    new = pd.DataFrame(columns=[
        'product_name', 'product_performance', 'components_of_computer'
    ])
    for row in benchmarks.itertuples():
        gpu_name = row.processed_name
        gpu_names_matched = gpu_names_in_db[gpu_names_in_db['product_name'.upper()] == gpu_name]
        if len(gpu_names_matched) == 0:
            continue
        new.loc[len(new)] = [
            *gpu_names_matched['product_name'.upper()].values, row.product_performance, 'gpu'
        ]
    return new

### gpu 이름 전처리하기
- 벤치마크 테이블의 gpu이름을 analytics.summary 테이블에 저장된 형태로 파싱
- 예) GeForce GTX 1650 Ti -> GTX1650Ti

In [10]:
def preprocess_gpu_names(gpu_benchmarks_row):
    remove_pattern = r'\d+GB'
    remove_keywords = ['GeForce', 'Radeon', 'NVDIA', 'Intel', 'Quadro', 'OEM', ' ']
    gpu_name = gpu_benchmarks_row['product_name']
    gpu_name = re.sub(remove_pattern, '', gpu_name)
    for remove_keyword in remove_keywords:
        gpu_name = gpu_name.replace(remove_keyword, '')
    gpu_name = gpu_name.replace('Arc', 'ARC')
    return gpu_name.strip()

### 크롤링한 벤치마크 정보 파일 전처리하여 저장하기
- analytics.summary에 저장되어있는 product_name의 값 형태로 cpu, gpu 이름들을 전처리하고,
- analytics.summary에 저장되어있는 product들의 벤치마크 정보만 추출하여 csv 파일로 저장한다.

In [15]:
def main():
    benchmarks_file = "../crawling/cpu_gpu_bench_marks.csv"
    product_names = pd.read_csv("../raw_data/cpu_gpu_products.csv")
    cpu_benchmarks = preprocess_cpu_benchmark(benchmarks_file, product_names)
    gpu_benchmarks = preprocess_gpu_benchmark(benchmarks_file, product_names)
    merged = pd.concat([cpu_benchmarks, gpu_benchmarks], ignore_index=True)
    merged = merged.sort_values(by=['product_performance'])
    merged = merged.drop_duplicates(subset=['product_name'], keep='last')
    merged = merged.sort_values(by=['product_name'], ignore_index=True)
    merged.to_csv("../raw_data/existing_cpu_gpu_benchmarks.csv")
    print("Preprocessing benchmarks done")
if __name__ == "__main__":
    main()

Preprocessing benchmarks done
